In [1]:
import os

In [2]:
%pwd

'd:\\Cdac_ML\\Assignments\\Assignment_4\\research'

In [3]:
os.chdir("../")

In [4]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataTransformationConfig:
    root_dir: Path
    data_path: Path

In [5]:
from src.Assignment_4.constants import *
from src.Assignment_4.utils.common import read_yaml, create_directories

class ConfigurationManager:
    def __init__(
        self,
        config_filepath = Config_yaml_path,
        params_filepath = params_yaml_path,
        schema_filepath = schema_yaml_path):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    
    def get_data_transformation_config(self) -> DataTransformationConfig:
        config = self.config.data_transformation

        create_directories([config.root_dir])

        data_transformation_config = DataTransformationConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
        )

        return data_transformation_config

In [8]:
from src.Assignment_4 import logger
import pandas as pd
from sklearn.model_selection import train_test_split

#update the component

import os
class DataTransformation:
    def __init__(self, config: DataTransformationConfig):
        self.config = config


    def train_test_spliting(self):
        data = pd.read_csv(r"D:\Cdac_ML\Assignments\Assignment_4\artifacts\data_ingestion\airplane.csv")

        # Split the data into training and test sets. (0.75, 0.25) split.
        train, test = train_test_split(data)

        train.to_csv(os.path.join(self.config.root_dir, "train.csv"),index = False)
        test.to_csv(os.path.join(self.config.root_dir, "test.csv"),index = False)

        logger.info("Splited data into training and test sets")
        logger.info(train.shape)
        logger.info(test.shape)

        print(train.shape)
        print(test.shape)

In [9]:
try:
    config = ConfigurationManager()
    data_transformation_config = config.get_data_transformation_config()
    data_transformation = DataTransformation(config=data_transformation_config)
    data_transformation.train_test_spliting()
except Exception as e:
    raise e

[2025-11-09 10:51:10,284: INFO: common: yaml file: D:\Cdac_ML\Assignments\Assignment_4\config\config.yaml loaded successfully]
[2025-11-09 10:51:10,287: INFO: common: yaml file: D:\Cdac_ML\Assignments\Assignment_4\params.yaml loaded successfully]
[2025-11-09 10:51:10,288: INFO: common: yaml file: D:\Cdac_ML\Assignments\Assignment_4\schema.yaml loaded successfully]
[2025-11-09 10:51:10,290: INFO: common: created directory at: artifacts]
[2025-11-09 10:51:10,290: INFO: common: created directory at: artifacts/data_transformation]
[2025-11-09 10:51:10,965: INFO: 1944876077: Splited data into training and test sets]
[2025-11-09 10:51:10,965: INFO: 1944876077: (97410, 24)]
[2025-11-09 10:51:10,965: INFO: 1944876077: (32470, 24)]
(97410, 24)
(32470, 24)
